# Pore analysis testing

In [ ]:
# Is porespy installed correctly?
import porespy as ps
import matplotlib.pyplot as plt
im = ps.generators.blobs(shape=[500, 500], porosity=0.6, blobiness=2)
plt.imshow(im)
plt.show()

In [ ]:
# General notebook settings
import seaborn
# Set seaborn theme
seaborn.set_theme(context='notebook', style='ticks')
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation

In [ ]:
# Let's look at an image from Alina

In [ ]:
import platform
import os

In [ ]:
if 'Win' in platform.system():
    Root = 'F:/'
else:
    Root = '/media/habi/Fast_SSD'
Path = os.path.join(Root, 'Schmid BFH Methylcellulose')
print('Our base path is %s' % Path)

In [ ]:
import imageio

In [ ]:
import glob

In [ ]:
# Load middle image from one scan

In [ ]:
Folder = os.path.join(Path, 'Blobs', 'MC-Titanweiss', 'rec')

In [ ]:
ImageList = sorted(glob.glob(os.path.join(Folder, '*.png')))

In [ ]:
print('Found %s images in %s' % (len(ImageList), Folder))
print('Loading the middle one below')

In [ ]:
# Load test image
inputimage = imageio.imread(ImageList[len(ImageList)//2], pilmode='L')

In [ ]:
# Show test image
plt.imshow(inputimage)
plt.title(inputimage.shape)
plt.show()

In [ ]:
# Use only central part of the image
crop = 666
plt.subplot(121)
croppedimage = inputimage[crop:-crop,crop:-crop]
plt.imshow(croppedimage)
plt.show()

In [ ]:
import skimage

In [ ]:
skimage.filters.gaussian?

In [ ]:
# Calculate a threshold to separate into foam and background
threshold_iso = skimage.filters.threshold_isodata(croppedimage)
threshold_otsu = skimage.filters.threshold_otsu(croppedimage)

In [ ]:
# Display gray value histogram of image
histogram = plt.hist(croppedimage.ravel(),
                     bins='doane', # nice bin size selection
                     histtype='bar',
                     log=True,
                     label='Histogram',
                     color=seaborn.color_palette()[0])
plt.axvline(threshold_iso, label='Isodata-Threshold@%s' % threshold_iso, c=seaborn.color_palette()[1])
plt.axvline(threshold_otsu, label='Otsu-Threshold@%s' % threshold_otsu, c=seaborn.color_palette()[2])
plt.legend()
plt.title('Logarithmic grayvalue histogram with %s bins' % len(histogram[1]))
seaborn.despine()
plt.show()

In [ ]:
binarizedimage = croppedimage < threshold_iso  # porespy expects 'True' for features of interest, so we true stuff smaller than the threshold, e.g the air

In [ ]:
plt.subplot(121)
plt.imshow(croppedimage)
plt.subplot(122)
plt.imshow(~binarizedimage) # Invert for displaying
plt.show()

In [ ]:
localthickness=ps.filters.local_thickness(binarizedimage)

In [ ]:
len(np.unique(localthickness))

In [ ]:
plt.subplot(131)
plt.imshow(localthickness)
plt.subplot(132)
plt.imshow(localthickness/binarizedimage)
plt.subplot(133)
plt.imshow(~binarizedimage, interpolation='auto')
plt.show()

In [ ]:
import scipy

In [ ]:
dt = scipy.ndimage.distance_transform_edt(binarizedimage)
distance = 250
peaks = skimage.feature.peak_local_max(dt, min_distance=distance)
# skeleton = skimage.morphology.skeletonize(skimage.morphology.dilation(binarizedimage))

In [ ]:
# plt.imshow(skeleton)

In [ ]:
plt.imshow(dt/binarizedimage, cmap='viridis')
for peak in peaks:
    plt.scatter(peak[1], peak[0], marker='x', c='white')
plt.title('Distance transformation with %s overlaid peaks > %s px' % (len(peaks), distance))
plt.show()

In [ ]:
pk = ps.filters.find_peaks(dt)

In [ ]:
crop = 800

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[6, 3])

pk = ps.filters.find_peaks(dt=dt)
ax[0].imshow((dt/binarizedimage)[crop:-crop,crop:-crop])
ax[0].axis(False)

ax[1].imshow(dt[crop:-crop,crop:-crop]/
             binarizedimage[crop:-crop,crop:-crop]/
             ~skimage.morphology.dilation(skimage.morphology.dilation(skimage.morphology.dilation(pk[crop:-crop,crop:-crop]))))
ax[1].axis(False);

In [ ]:
np.unique(skimage.morphology.dilation(pk))

In [ ]:
crop = 1200

In [ ]:
plt.imshow(localthickness[crop:-crop,crop:-crop]/binarizedimage[crop:-crop,crop:-crop], interpolation='none')
plt.show()

In [ ]:
plt.imshow(localthickness[crop:-crop,crop:-crop]/binarizedimage[crop:-crop,crop:-crop], cmap='viridis')
plt.show()

Pore size distribution as per https://porespy.org/examples/metrics/reference/pore_size_distribution.html

In [ ]:
data = ps.metrics.pore_size_distribution(im=localthickness)
fig, ax = plt.subplots(1, 3, figsize=[10, 4])
ax[0].plot(data.bin_centers, data.pdf)
ax[1].plot(data.bin_centers, data.cdf)
ax[2].bar(data.bin_centers, data.cdf, data.bin_widths, edgecolor='k')
ax[0].set_title("Probability Density Function")
ax[1].set_title("Cumulative Density Function")
ax[2].set_title('Bar Plot');

In [ ]:
data = ps.metrics.pore_size_distribution(im=localthickness, bins=100)
fig, ax = plt.subplots(1, 3, figsize=[10, 4])
ax[0].plot(data.bin_centers,data.pdf)
ax[1].plot(data.bin_centers,data.cdf)
ax[2].bar(data.bin_centers, data.cdf, data.bin_widths, edgecolor='k')
ax[0].set_title("Probability Density Function")
ax[1].set_title("Cumulative Density Function")
ax[2].set_title('Bar Plot');

In [ ]:
data = ps.metrics.pore_size_distribution(im=localthickness, log=False)
fig, ax = plt.subplots(1, 3, figsize=[10, 4])
ax[0].plot(data.bin_centers,data.pdf)
ax[1].plot(data.bin_centers,data.cdf)
ax[2].bar(data.bin_centers, data.cdf, data.bin_widths, edgecolor='k')
ax[0].set_title("Probability Density Function")
ax[1].set_title("Cumulative Density Function")
ax[2].set_title('Bar Plot');

In [ ]:
voxel_size=1.5e-3
data = ps.metrics.pore_size_distribution(im=localthickness, voxel_size=voxel_size, log=True)
fig, ax = plt.subplots(1, 2, figsize=[10, 4])
ax[0].bar(data.bin_centers, data.cdf, data.bin_widths, edgecolor='k');
ax[0].set_title("PDF scaling included in the result");
data = ps.metrics.pore_size_distribution(im=im, log=False)
ax[1].bar(data.bin_centers*voxel_size, data.cdf, data.bin_widths*voxel_size, edgecolor='k');
ax[1].set_title("PDF scaling after the result");

In [ ]:
print(data)

In [ ]:
print(data.bin_centers)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
ax.set_xlabel('log(Pore Radius) [voxels]')
ax.set_ylabel('Normalized Volume Fraction')
ax.bar(x=psd.LogR, height=psd.pdf, width=psd.bin_widths, edgecolor='k');

In [ ]:
import numpy as np

In [ ]:
from skimage.morphology import binary_dilation
ps.visualization.set_mpl_style()
np.random.seed(1)

In [ ]:
im = croppedimage < threshold_iso

In [ ]:
# im = ps.generators.overlapping_spheres([500, 500], r=10, porosity=0.5)
fig, ax = plt.subplots()
ax.imshow(im, origin='lower');

In [ ]:
snow_out = ps.filters.snow_partitioning(im, r_max=4, sigma=0.4)
print(snow_out)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=[8, 8])
ax[0, 0].imshow(snow_out.im)
ax[0, 1].imshow(snow_out.dt/binarizedimage)
dt_peak = snow_out.dt.copy()
peaks_dilated = binary_dilation(snow_out.peaks > 0)
dt_peak[peaks_dilated > 0] = np.nan
ax[1, 0].imshow(dt_peak/binarizedimage)
ax[1, 1].imshow(ps.tools.randomize_colors(snow_out.regions)/binarizedimage)
ax[0, 0].set_title("Binary image");
ax[0, 1].set_title("Distance transform");
ax[1, 0].set_title("Distance transform peaks");
ax[1, 1].set_title("Segmentation");

In [ ]:
from skimage.io import imread_collection

In [ ]:
seq = imread_collection('/media/habi/Fast_SSD/Schmid BFH Methylcellulose/Blobs/MC-Titanweiss/rec/TiO_rec00002[12]*.png')

In [ ]:
len(seq.files)

In [ ]:
from matplotlib.pyplot import subplots

In [ ]:
crop=100

In [ ]:
fig, ax = subplots(figsize=[5, 5])
ax.imshow(seq[0][1000+crop:-800-crop,600+crop:-1200-crop,0])

In [ ]:
im3d = np.zeros([*seq[0][1000+crop:-800-crop,600+crop:-1200-crop,0].shape, len(seq)])
for i, im in enumerate(seq):
    im3d[..., i] = im[1000+crop:-800-crop,600+crop:-1200-crop,0]

In [ ]:
im3d.shape

In [ ]:
ps.visualization.show_planes?

In [ ]:
ps.visualization.show_panels(im3d, axis=2)

In [ ]:
plt.imshow(ps.visualization.show_planes(im3d, spacing=150), cmap='magma')
plt.show()

In [ ]:
fig, ax = subplots(figsize=[5, 5])
ax.hist(im3d.flatten(), bins='doane', log=True);

In [ ]:
fig, ax = subplots(figsize=[5, 5])
ax.imshow(im3d[..., 0] < threshold_otsu);

In [ ]:
im = im3d < threshold_otsu
fig, ax = subplots(figsize=[5, 5])
ax.imshow(ps.visualization.sem(im, axis=2), cmap='grey');

In [ ]:
import numpy as np
import porespy as ps
import scipy.ndimage as spim
import matplotlib.pyplot as plt
ps.visualization.set_mpl_style()
np.random.seed(1)

In [ ]:
im = ps.generators.blobs(shape=[200, 200], porosity=0.6, blobiness=1)
fig, ax = plt.subplots()
ax.imshow(im, cmap=plt.cm.inferno);

In [ ]:
snow = ps.filters.snow_partitioning(im=im)
regions = snow.regions*snow.im

In [ ]:
fig, ax = plt.subplots()
ax.imshow(regions);

In [ ]:
props = ps.metrics.regionprops_3D(regions)

In [ ]:
r = props[0]
attrs = [a for a in r.__dir__() if not a.startswith('_')]
print(attrs)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(r.image);

In [ ]:
fig, ax = plt.subplots()
ax.imshow(r.border + 0.5*r.inscribed_sphere);

In [ ]:
fig, ax = plt.subplots()
ax.imshow(r.image + 1.0*r.convex_image);

In [ ]:
print(f"Solidity: {r.solidity:.3f}")

In [ ]:
df = ps.metrics.props_to_DataFrame(props)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3))
ax[0].hist(df['volume'], edgecolor="k")
ax[1].hist(df['solidity'], edgecolor="k")
ax[2].hist(df['sphericity'], edgecolor="k");
ax[0].set_title("Volume")
ax[1].set_title("Solidity")
ax[2].set_title("Sphericity");

In [ ]:
df.iloc[0]

In [ ]:
# Create an image of maximally inscribed spheres
sph = ps.metrics.prop_to_image(regionprops=props, shape=im.shape, prop='inscribed_sphere')
fig, ax = plt.subplots()
ax.imshow(sph + 0.5*(~im) , cmap=plt.cm.inferno);

In [ ]:
# Create an image colorized by solidity
sph = ps.metrics.prop_to_image(regionprops=props, shape=im.shape, prop='solidity')
fig, ax = plt.subplots()
ax.imshow(sph + 0.5*(~im) , cmap='viridis');